In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
global gama
states = 100
actions = 4
reward = -0.01
gama = 0
world = np.zeros((states, actions))

pits = []
j = 0
while True:
    p = np.random.randint(1,99)
    if p not in pits:
        pits.append(p)
        j+=1
        if j > 10:
            break

wins = [0,99]

In [ ]:
sess = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, states], name='input_states')
y_ = tf.placeholder(tf.float32, shape=[None, actions], name='correct_answer')
W = tf.Variable(tf.zeros([states ,actions]), name='weights')
b = tf.Variable(tf.zeros([actions]), name='bias')

# predicted action
y = tf.matmul(x,W) + b
y_soft = tf.nn.softmax(y)

cross_antropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_antropy)

init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
def get_next_state(state, gama):
    # act greedy with random
    coin = np.random.randint(500 + gama)
    a = get_best_action_from_nn(state)
    if coin < 400:
        random_actions = []
        if x in actions:
            if x !=a:
                random_actions.append(x)
        a = np.random.randint(random_actions)
    s = get_next_valid_state(state, a)

    return s, a

In [ ]:
LEFT = 0
RIGHT = 1
DOWN = 2
UP = 3

def get_possible_actions(state):
    x = int(state % np.sqrt(states))
    y = int(state // np.sqrt(states))
    actions=[]
    if x > 0:
        actions.append(LEFT)
        if x < (np.sqrt(states) - 1):
            actions.append(RIGHT)        
    elif x < (np.sqrt(states) - 1):
            actions.append(RIGHT)

    if y > 0:
        actions.append(DOWN)
        if y < (np.sqrt(states) - 1):
            actions.append(UP)
    elif y < (np.sqrt(states) - 1):
            actions.append(UP)
    return actions

In [ ]:
def get_next_valid_state(state, action):
    x = int(state % np.sqrt(states))
    y = int(state // np.sqrt(states))   
    if x > 0 and action == LEFT:
        return state - 1
    if x < 9 and action == RIGHT:
        return state + 1
    if y > 0 and action == DOWN:
        return state - int(np.sqrt(states)) 
    if y < 9 and action == UP:
        return state + int(np.sqrt(states))
    return state

In [ ]:
def get_current_reward(state):
    if state in wins:
        return 1
    if state in pits:
        return 0
    return 0.5

In [ ]:
def get_state_as_array(state):
    s = np.zeros(states, dtype=np.float32)
    s[state] = 1.0
    return s

In [ ]:
def get_reward_from_nn(state, a):
    s = get_state_as_array(state)
    b = sess.run(y, feed_dict={x: [s]})
    return b[0,a]

In [ ]:
def get_best_action_from_nn(state):
    s = get_state_as_array(state)
    a = sess.run(tf.arg_max(y,1), feed_dict={x: [s]})
    return a[0]    

In [ ]:
def get_max_reward_from_nn(state):
    s = get_state_as_array(state)
    m = sess.run(tf.reduce_max(y,1), feed_dict={x: [s]})
    return m[0]

In [ ]:
def get_all_rewards(state):
    s = get_state_as_array(state)
    b = sess.run(y, feed_dict={x: [s]})
    return b[0].tolist()    

In [ ]:
def train(state, rewards):
    s = get_state_as_array(state)
    for i in range(5):
        sess.run(train_step, feed_dict={x:[s], y_:[rewards]})

In [ ]:
def get_first_state():
    found = False
    while not found:
        state = np.random.randint(states)
        if state not in wins and state not in pits:
            return state
            

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def update_rewards(state):
    possible_actions = get_possible_actions(state)
    all_rewards = get_all_rewards(state)

    for action in range(actions):
        if action in possible_actions:
            next_state = get_next_valid_state(state, action)
            reward = get_max_reward_from_nn(next_state)
            all_rewards[action] = reward - 0.1
        else:
            all_rewards[action] = 0
#     all_rewards = softmax(all_rewards)
    return all_rewards

In [ ]:
state = get_first_state()
state

In [ ]:


for i in range(20):
    
    # update NN
    all_rewards = update_rewards(state)
    train(state, all_rewards)
    
    ls = state
    # Act greedy
    state, la = get_next_state(state, gama)

    # get current state value
    r = get_current_reward(state)

    # update privious state and action
    rewards = update_rewards(ls)

    if r != 0.5:
        rewards[la] = r

#     rewards = softmax(rewards)

    # update privious state
    train(ls, rewards)
    gama+=1


In [ ]:
board = ["" for i in range(100)]

for i in range(states):
        a = get_best_action_from_nn(i)
        if a == LEFT:
            board[i] = '<'
        elif a == RIGHT:
            board[i] = '>'
        elif a == DOWN:
            board[i] = '^'
        else:
            board[i] = 'V'

for p in pits:
    board[p] = 'O'
board[99] = 'X'
board[0] = 'X'
board = np.reshape(board, (10,10))


board

In [ ]:
# ls = 24
# la = LEFT

# state = 23

for k in range(2000):


    cr = get_current_reward(state)

    pa = get_possible_actions(ls)
    rr = []
    for i in range(actions):
        if i in pa:
            s1 = get_next_valid_state(ls, i)
            r1 = get_max_reward_from_nn(s1)
            rr.append(r1 - 0.1)
        else:
            rr.append(0)
    if cr !=0.5:
        rr[la] = cr

    # print ('%s: %s\n' % (ls,rr))

    s = get_state_as_array(ls)
    for i in range(50):
        sess.run(train_step, feed_dict={x:[s], y_:[rr]})

    ls = state
    while True:
        state, la = get_next_state(state, 100)
        if state != ls:
            break

    

In [ ]:
ww = sess.run(W, feed_dict={x:[s], y_:[rr]})
bb = sess.run(b, feed_dict={x:[s], y_:[rr]})
# ww, bb

In [ ]:
# update NN
all_rewards = update_rewards(state)
train(state, all_rewards)

In [ ]:
ls = state
# Act greedy
state, la = get_next_state(state, gama)
ls, la, state

In [ ]:
# get current state value
r = get_current_reward(state)
r

In [ ]:
# update privious state and action
rewards = update_rewards(ls)

if r != 0.5:
    rewards[la] = r

#     rewards = softmax(rewards)

# update privious state
train(ls, rewards)
gama+=1
ls, la, rewards